In [1]:
from codebase.classes import Data, Particles
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, run_stan_model
from run_ibis import run_ibis

from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [2]:
def form_df(samples, num_chains, rows):
    dfs = []
    for cn in range(num_chains):
        for r in range(rows):
            if rows>1:
                df = pd.DataFrame(
                        samples[:,cn, r]
                )
            else:
                df = pd.DataFrame(
                        samples[:,cn]
                )
            df.insert(0, 'idx', np.arange(df.shape[0]))
            df = df.melt(id_vars ='idx', var_name = 'col')
            df.insert(1, 'row' , r)
            df.insert(1, 'chain', cn)
            dfs.append(df)
    return pd.concat(dfs).reset_index(drop=True)


def get_post_df(ps, param):
    samples = ps[param]
    mcmc_iter = samples.shape[0]
    num_chains = samples.shape[1]
    if samples.ndim > 3:
        rows = samples.shape[2]
        cols = samples.shape[3]
        df = form_df(samples, num_chains, rows)
    else:
        cols = samples.shape[2]
        rows = 1
        df = form_df(samples, num_chains, 1)
    return df


def plot_density(df, width=300, height=50) :
    return alt.Chart(df).transform_fold(
        ['value']
    ).transform_density(
        density='value',
        groupby=['cn', 'row', 'col', 'source'],
    ).mark_area(opacity=0.5).encode(
        alt.X('value:Q'),
        alt.Y('density:Q'),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
    ).resolve_scale(
        x='independent'
    ).properties(width=width, height=height)



## 2 factor Sim EZ

In [3]:
existing_directory = None
task_handle = 'ez_comp'
gen_model = 0

if existing_directory is None:
    log_dir = make_folder(task_handle)  
    print("\n\nCreating new directory: %s" % log_dir)

else:
    log_dir = existing_directory
    log_dir = path_backslash(log_dir)
    print("\n\nReading from existing directory: %s" % log_dir)




Creating new directory: ./log/20201218_172340_ez_comp/


## Create Data

In [4]:
# generate data
exp_data = Data(
    name = task_handle, 
    model_num = 4, 
    size = 100,
    random_seed = 6
    )
    
exp_data.generate()

## Run HMC

In [5]:
sm = load_obj('sm', 'log/compiled_models/model1/')

fit_run = run_stan_model(
    data = exp_data.get_stan_data(),
    compiled_model = sm,
    num_samples = 1000, 
    num_warmup = 1000,
    num_chains = 1,
    adapt_engaged=True
    )

In [6]:
param_names = ['beta', 'alpha', 'Marg_cov']

ps = fit_run.extract(
        permuted=False, pars=param_names)

save_obj(ps, 'ps_hmc', log_dir)

## Run IBIS

In [7]:
# load existing results

# log_dir = 'log/20201218_172340_ez_comp/'
# particles = load_obj('particles', log_dir)

In [8]:
gen_model = 0
model_num = 1

ibis = run_ibis(
    exp_data,
    model_num,
    1000,
    gen_model,
    log_dir
    )

for name in ['alpha', 'Marg_cov']:
    samples = np.squeeze(ibis['particles'].particles[name])
    w = exp_and_normalise(ibis['particles'].weights)
    print('\n\nEstimate')
    print(np.round(np.average(samples,axis=0, weights=w),2))
    # print('\nRead Data')
    # print(np.round(exp_data.raw_data[name],2))

100%|██████████| 100/100 [45:09<00:00, 27.09s/it]




Marginal Likelihood 0.10378


Estimate
[ 0.   -0.02  0.06 -0.05 -0.05  0.01]


Estimate
[[0.83 0.29 0.32 0.05 0.05 0.04]
 [0.29 0.71 0.37 0.06 0.06 0.04]
 [0.32 0.37 0.86 0.06 0.07 0.05]
 [0.05 0.06 0.06 0.92 0.4  0.28]
 [0.05 0.06 0.07 0.4  0.99 0.32]
 [0.04 0.04 0.05 0.28 0.32 0.81]]


## Plot Both results

In [10]:
# have to resample particles to get rid of weights
particles = ibis['particles']
particles.resample_particles()

In [12]:
param = 'beta'
df = get_post_df(ps, param) 
df['source'] = 'hmc'
df2 = get_post_df(particles.particles, param)
df2['source'] = 'ibis'

plot_density(pd.concat([df,df2]))

alt.Chart(...)

In [16]:
param = 'alpha'
df = get_post_df(ps, param) 
df['source'] = 'hmc'
df2 = get_post_df(particles.particles, param)
df2['source'] = 'ibis'

plot_density(pd.concat([df,df2]))


alt.Chart(...)